# What this file does

Scrapes the certificates of need from the Office of Health Strategy's website. Looking for applications for mental health facilities over time.

In [38]:
#tools
import requests
import pandas as pd
from datetime import datetime
import numpy as np

In [39]:
#copy cURL and use https://curl.trillworks.com/
cookies = {
    '__utma': '64328189.1312055381.1587758998.1591459165.1592333659.2',
    '_ga_76Z1SF2P6H': 'GS1.1.1613926003.2.0.1613926003.0',
    '_ga_YVLSZ2433X': 'GS1.1.1617317259.7.0.1617317259.0',
    '_ga_M5NRCCSVBB': 'GS1.1.1617994491.6.0.1617994491.0',
    '_hjid': '3adbc369-236b-4856-9e04-82f1c72ece59',
    '_ga_Z8GFYXH9YT': 'GS1.1.1623862014.17.0.1623862014.0',
    '_hjTLDTest': '1',
    '_gid': 'GA1.2.685151373.1624893575',
    '_ga': 'GA1.2.1312055381.1587758998',
    '_ga_89NN4475GK': 'GS1.1.1624995736.3.0.1624995963.0',
}

headers = {
    'Connection': 'keep-alive',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'Accept': '*/*',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://dphconwebportal.ct.gov',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://dphconwebportal.ct.gov/Report',
    'Accept-Language': 'en-US,en;q=0.9',
}


In [40]:
#page loops, pandas append
i=1
for i in range(1,33):
    data = {
      'sort': '',
      'page': f"{str(i)}",
      'pageSize': '15',
      'group': '',
      'filter': ''
    }    
    response = requests.post('https://dphconwebportal.ct.gov/Report/UserApplications_Read', headers=headers, cookies=cookies, data=data)
    result = response.json()
    if i ==1:
        df_1 = pd.DataFrame(result["Data"])
    else:
        df = pd.DataFrame(result["Data"])
        df_1 = df_1.append(df)
    i += 1
    

In [41]:
#what's the payment column about
df_1.columns

Index(['ApplicationId', 'UserId', 'UserName', 'ApplicationType', 'Docket',
       'DocketType', 'DocketStatus', 'DocketStatusDate', 'Status',
       'CreateDate', 'Payment', 'PaymentStatus', 'Applicant', 'Title',
       'Expenditure', 'Village', 'ListofDocuments'],
      dtype='object')

In [42]:
#go from UNIX to datetime to month/year
df_1['DocketStatusDate'] = df_1['DocketStatusDate'].str.extract(r'.Date\((.*)\).')
df_1['CreateDate'] = df_1['CreateDate'].str.extract(r'.Date\((.*)\).')

df_1['DocketStatusDate'] = pd.to_datetime(df_1['DocketStatusDate'], unit='ms')
df_1['CreateDate'] = pd.to_datetime(df_1['CreateDate'], unit='ms')

df_1['CreateDate_MonthYear'] = df_1['CreateDate'].dt.to_period('M')
df_1['CreateDate_Year'] = df_1['CreateDate'].dt.to_period('Y')

In [100]:
#data goes only as far back as 2017, October
df_1['CreateDate_Year'].unique()
df_1['CreateDate_MonthYear'].unique()

<PeriodArray>
['2021-03', '2021-04', '2021-06', '2021-05', '2021-02', '2021-01', '2020-12',
 '2020-10', '2020-11', '2020-09', '2020-08', '2020-07', '2020-06', '2020-05',
 '2020-04', '2020-03', '2020-02', '2020-01', '2019-12', '2019-11', '2019-10',
 '2019-09', '2019-08', '2019-07', '2019-06', '2019-05', '2019-04', '2019-03',
 '2019-02', '2019-01', '2018-12', '2018-11', '2018-10', '2018-09', '2018-08',
 '2018-07', '2018-06', '2018-05', '2018-04', '2018-03', '2018-02', '2018-01',
 '2017-12', '2017-11', '2017-09', '2017-10', '2017-08']
Length: 47, dtype: period[M]

In [81]:
#get all CON applications to do with mental health
for con in list(df_1['Title'].unique()):
    if "mental health" in con:
        print('"'+con+'"'+',')

"Establishing a new mental health care facility                             ",
"Transfer of Ownership of a mental health facility",
"Establishment of mental health day treatment facility and psychiatric outpatient clinic for adults.  Requested additional information from petitioner on 8/2/16.",
"Relocation of mental health residential living center",
"Establishment of mental health and substance abuse facility.",
"SELF Inc. seeks to establish a free standing mental health residential living center in Wolcott, CT. Requested additional information from petitioner on 2/1/16.",
"Establishment of Shoreline Wellness Behavioral Health Clinic, LLC to provide mental health services in West Haven.",


In [88]:
#filter
df_mental_health = df_1[df_1['Title'].isin(["Establishing a new mental health care facility                             ",
"Transfer of Ownership of a mental health facility",
"Establishment of mental health day treatment facility and psychiatric outpatient clinic for adults.  Requested additional information from petitioner on 8/2/16.",
"Relocation of mental health residential living center",
"Establishment of mental health and substance abuse facility.",
"SELF Inc. seeks to establish a free standing mental health residential living center in Wolcott, CT. Requested additional information from petitioner on 2/1/16.",
"Establishment of Shoreline Wellness Behavioral Health Clinic, LLC to provide mental health services in West Haven."])]

In [89]:
#plot over time by year
pd.DataFrame(df_mental_health.groupby(by='CreateDate_Year')['Title'].value_counts())

Title
CreateDate_Year Title                                                    
2017            Establishing a new mental health care facility ...      2
2018            Establishing a new mental health care facility ...     15
                Establishment of Shoreline Wellness Behavioral ...      1
                Establishment of mental health and substance ab...      1
                Establishment of mental health day treatment fa...      1
                Relocation of mental health residential living ...      1
                SELF Inc. seeks to establish a free standing me...      1
                Transfer of Ownership of a mental health facility       1
2019            Establishing a new mental health care facility ...      7
                Transfer of Ownership of a mental health facility       3
2020            Establishing a new mental health care facility ...     17
                Transfer of Ownership of a mental health facility       1
2021            Establishing a new mental health care facility ...      6
                Transfer of Ownership of a mental health facility       1

In [94]:
#where are these? 
df_mental_health[df_mental_health['CreateDate_Year']>'2018']['Village'].value_counts()

Middletown      5
Hartford        3
New Canaan      2
Summit          2
Los Alamitos    2
Arlington       2
Madison         2
Manchester      2
Waterbury       1
Greenwich       1
Derby           1
Stratford       1
Milford         1
Enfield         1
West Haven      1
New City        1
New Britain     1
Canaan          1
New Haven       1
New Haven       1
Denver          1
Naxhville       1
Danbury         1
Name: Village, dtype: int64

In [96]:
#a lot of community health center in here post 2018
df_mental_health['Applicant'].value_counts()

Community Health Center, Inc.                                                                           4
Mountainside Treatment Center                                                                           2
Continuum of Care                                                                                       2
American Day CD Centers, LLC                                                                            2
Shoreline Wellness Center , LLC                                                                         2
Column Health                                                                                           2
CT Renaissance                                                                                          2
center for discovery                                                                                    2
Walden Behavioral Care , LLC                                                                            1
The Center for Christian Inc.                 

In [101]:
df_1.to_csv('all_cons.csv')